# 모델
- skt/A.X-4.0

# 단계별 전체 흐름
---
1. 각 파일별로 텍스트 chunk → 벡터DB(FAISS 등) 저장
---
2. 질문 입력
    - → ex) "두피에 하얗게 벗겨지고 가려워요. 무슨 질환일까요?"
---
3. 벡터DB에서 chunk 검색(Top-k 유사도 검색)
    - → 질문 임베딩 → 벡터DB에서 가장 유사한 chunk 반환
---
4. 검색 결과와 사용자 질문을 EXAONE 모델로 전달
    - → Prompt에 ‘검색 결과 + 원 질문’ 조합하여 입력
---
5. EXAONE이 최종 답변 생성
---

# 1. 임포트 & 데이터 폴더 지정

In [1]:
# 데이터 처리 및 파일 입출력
import os
import pandas as pd
import glob

# RAG, 임베딩, 벡터DB 관련 예시 임포트 (필요에 따라 사용)
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM

# 텍스트 전처리, 토큰화 등 (필요에 따라)
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import re

# openai 호출
from openai import OpenAI

# 마크다운 글씨 굵게 하기
from IPython.display import display, Markdown

print(torch.cuda.is_available())
print(torch.cuda.device_count())

# 파일 경로 자동 지정
data = "../dataset"

True
1


# 2. dataset 폴더 내 모든 CSV 파일 읽어오기

In [2]:
df_1200 = pd.read_csv(os.path.join(data, "1200_v1.csv"))
df_amc = pd.read_csv(os.path.join(data, "amc.csv"))
df_daily = pd.read_csv(os.path.join(data, "daily_dataset.csv"))
df_final = pd.read_csv(os.path.join(data, "final_v7.csv"))
df_kdca = pd.read_csv(os.path.join(data, "kdca.csv"))
df_snu = pd.read_csv(os.path.join(data, "snu.csv"))

# 데이터 미리보기 (각 파일별)
print("1200_v1.csv:", df_1200.shape, "\n", df_1200.head(2), "\n")
print("amc.csv:", df_amc.shape, "\n", df_amc.head(2), "\n")
print("daily_dataset.csv:", df_daily.shape, "\n", df_daily.head(2), "\n")
print("final_v7.csv:", df_final.shape, "\n", df_final.head(2), "\n")
print("kdca.csv:", df_kdca.shape, "\n", df_kdca.head(2), "\n")
print("snu.csv:", df_snu.shape, "\n", df_snu.head(2), "\n")

1200_v1.csv: (1200, 2) 
   label                                               text
0    건선  지난 몇 주 동안 팔, 다리, 몸통에 피부 발진이 생겼어요. 빨갛고 가려운데, 마른...
1    건선  제 피부가 벗겨지고 있어요, 특히 무릎, 팔꿈치, 그리고 두피 쪽이요. 이 벗겨짐이... 

amc.csv: (1278, 7) 
                                 병명  \
0   18번 염색체 단완결실 증후군(18p monosomy)   
1  18번 염색체 장완결실 증후군 (18q monosomy)   

                                                  정의  \
0  18번 염색체 단완결실 증후군은 흔하지 않지만 특징적인 표현형을 가지고 있는 질환을...   
1  18번 염색체 장완결실 증후군은 흔한 염색체 이상 질환은 아니지만, 특징적인 표현형...   

                                                  원인  \
0  대부분 염색체 검사를 통해 18번 염색체 단완결실이 확인됩니다. 이 경우 환자의 8...   
1  18번 염색체 장완결실 증후군은 대부분 자연발생적으로 발생합니다. 나머지는 18번 ...   

                                                  증상  \
0  18번 염색체 단완결실 증후군에는 소두증, 안검 하수, 사시, 양안 격리증, 크고 ...   
1  18번 염색체 장완결실 증후군은 소두증, 안면 중앙부 저형성, 깊게 패인 눈, 짧은...   

                                                  진단  \
0  18번 염색체 단완결실 증후군은 일반적으로 세포 유전학적 검사(말초혈액 염색체 검사...   
1  18번 염색체 장완결실 증후군의 진단은 일반적으로 세포 유전학적 검사

# 3. 각 파일별로 chunking → 벡터DB 저장

In [3]:
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")


def chunk_text(texts, chunk_size=300):
    """간단하게 문자열을 일정 길이로 자르는 함수"""
    chunks = []
    for text in texts:
        if pd.isnull(text):
            continue
        for i in range(0, len(text), chunk_size):
            chunk = text[i : i + chunk_size]
            if chunk.strip():
                chunks.append(chunk)
    return chunks


### 1. 1200_v1.csv
df_1200 = pd.read_csv(os.path.join(data, "1200_v1.csv"))
texts_1200 = df_1200["label"].astype(str) + "\n" + df_1200["text"].astype(str)
chunks_1200 = chunk_text(texts_1200.tolist(), chunk_size=300)
db_1200 = FAISS.from_texts(chunks_1200, embedding=embedding_model)
db_1200.save_local("faiss_db_1200_v1")

### 2. amc.csv
df_amc = pd.read_csv(os.path.join(data, "amc.csv"))
# '병명', '정의', '원인', '증상', '진단', '치료', '진료과' 모두 하나로 합치기 (필요한 부분만 골라도 됨)
amc_texts = (
    df_amc["병명"].astype(str)
    + "\n"
    + df_amc["정의"].astype(str)
    + "\n"
    + df_amc["원인"].astype(str)
    + "\n"
    + df_amc["증상"].astype(str)
    + "\n"
    + df_amc["진단"].astype(str)
    + "\n"
    + df_amc["치료"].astype(str)
)
chunks_amc = chunk_text(amc_texts.tolist(), chunk_size=400)
db_amc = FAISS.from_texts(chunks_amc, embedding=embedding_model)
db_amc.save_local("faiss_db_amc")

### 3. daily_dataset.csv
df_daily = pd.read_csv(os.path.join(data, "daily_dataset.csv"))
daily_texts = df_daily["증상"].astype(str) + "\n" + df_daily["일상말"].astype(str)
chunks_daily = chunk_text(daily_texts.tolist(), chunk_size=250)
db_daily = FAISS.from_texts(chunks_daily, embedding=embedding_model)
db_daily.save_local("faiss_db_daily_dataset")

### 4. final_v7.csv
df_final = pd.read_csv(os.path.join(data, "final_v7.csv"))
final_texts = df_final["label"].astype(str) + "\n" + df_final["text"].astype(str)
chunks_final = chunk_text(final_texts.tolist(), chunk_size=300)
db_final = FAISS.from_texts(chunks_final, embedding=embedding_model)
db_final.save_local("faiss_db_final_v7")

### 5. kdca.csv
df_kdca = pd.read_csv(os.path.join(data, "kdca.csv"))
kdca_texts = (
    df_kdca["병명"].astype(str)
    + "\n"
    + df_kdca["정의"].astype(str)
    + "\n"
    + df_kdca["원인"].astype(str)
    + "\n"
    + df_kdca["증상"].astype(str)
    + "\n"
    + df_kdca["진단"].astype(str)
    + "\n"
    + df_kdca["치료"].astype(str)
)
chunks_kdca = chunk_text(kdca_texts.tolist(), chunk_size=400)
db_kdca = FAISS.from_texts(chunks_kdca, embedding=embedding_model)
db_kdca.save_local("faiss_db_kdca")

### 6. snu.csv
df_snu = pd.read_csv(os.path.join(data, "snu.csv"))
snu_texts = (
    df_snu["병명"].astype(str)
    + "\n"
    + df_snu["정의"].astype(str)
    + "\n"
    + df_snu["원인"].astype(str)
    + "\n"
    + df_snu["증상"].astype(str)
    + "\n"
    + df_snu["진단/검사"].astype(str)
    + "\n"
    + df_snu["치료"].astype(str)
)
chunks_snu = chunk_text(snu_texts.tolist(), chunk_size=400)
db_snu = FAISS.from_texts(chunks_snu, embedding=embedding_model)
db_snu.save_local("faiss_db_snu")

C:\Users\user\AppData\Local\Temp\ipykernel_23796\4244311411.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttenti

# 4-1. RAG(검색+생성) 파이프라인으로 작업

In [6]:
# [1] 사용자 질문 입력
user_question = input("질문을 입력하세요: ")

# [2] 여러 벡터DB 모두 로드
db_paths = [
    "faiss_db_1200_v1",
    "faiss_db_amc",
    "faiss_db_daily_dataset",
    "faiss_db_final_v7",
    "faiss_db_kdca",
    "faiss_db_snu",
]
db_list = [
    FAISS.load_local(db_path, embedding_model, allow_dangerous_deserialization=True)
    for db_path in db_paths
]

# [3] 각 DB에서 Top-k 검색, 결과 합치기 (속도↑, top_k=1)
top_k = 1
retrieved_chunks = []
for db in db_list:
    docs = db.similarity_search(user_question, k=top_k)
    retrieved_chunks.extend([doc.page_content for doc in docs])

# [4] context(검색 결과) 텍스트로 결합 (전체 길이 제한)
retrieved_context = "\n---\n".join(retrieved_chunks)
max_context_length = 600
retrieved_context = retrieved_context[:max_context_length]

# [5] 프롬프트 생성 (딱 3문장만, 안내/예시 금지)
prompt = f"""
아래 환자 증상에 대해 반드시 1~3번 형식으로만 딱 한 번 출력하세요.
절대 반복하거나 설명하지 말고, 줄바꿈 없이 한 줄로 출력하세요.

1. 예상되는 병명(2~3가지):
2. 추천 진료과(1줄):
3. 예방 및 관리 방법(2줄 이내):

[증상 정보]
{retrieved_context}
"""

# [6] SKT A.X-4.0 API로 답변 생성 (OpenAI 스타일)
client = OpenAI(
    base_url="https://guest-api.sktax.chat/v1",
    api_key="sktax-XyeKFrq67ZjS4EpsDlrHHXV8it"  # 여기에 발급받은 키를 넣으세요
)

response = client.chat.completions.create(
    model="ax4",  # skt/A.X-4.0 API 전용 모델명 (주의: "skt/A.X-4.0" 아님)
    messages=[
        {"role": "user", "content": prompt}
    ]
)
answer = response.choices[0].message.content.strip()

# [7] 번호 답변만 추출 (정규표현식)
matches = re.findall(r"1\..*?\n2\..*?\n3\..*(?=\n|$)", answer, flags=re.DOTALL)
answer_only = matches[-1].strip() if matches else answer

print("\n[AX-4.0 답변]\n")
display(Markdown(answer_only))


[AX-4.0 답변]



1. 감기, 부비동염, 폐렴
2. 내분비내과(내분비 질환 체크 후 필요시 내과 또는 호흡기내과로 전과)
3. 손 씻기 습관화, 충분한 수분 섭취와 실내 습도 조절 유지

In [ ]:
# [1] 사용자 질문 입력
user_question = input("질문을 입력하세요: ")

# [2] 여러 벡터DB 모두 로드
db_paths = [
    "faiss_db_1200_v1",
    "faiss_db_amc",
    "faiss_db_daily_dataset",
    "faiss_db_final_v7",
    "faiss_db_kdca",
    "faiss_db_snu",
]
db_list = [
    FAISS.load_local(db_path, embedding_model, allow_dangerous_deserialization=True)
    for db_path in db_paths
]

# [3] 각 DB에서 Top-k 검색, 결과 합치기 (속도↑, top_k=1)
top_k = 1
retrieved_chunks = []
for db in db_list:
    docs = db.similarity_search(user_question, k=top_k)
    retrieved_chunks.extend([doc.page_content for doc in docs])

# [4] context(검색 결과) 텍스트로 결합 (전체 길이 제한)
retrieved_context = "\n---\n".join(retrieved_chunks)
max_context_length = 600
retrieved_context = retrieved_context[:max_context_length]

# [5] 프롬프트 생성 (딱 3문장만, 안내/예시 금지)
prompt = f"""
아래 환자 증상에 대해 반드시 1~3번 형식으로만 딱 한 번 출력하세요.
절대 반복하거나 설명하지 말고, 줄바꿈 없이 한 줄로 출력하세요.

1. 예상되는 병명(2~3가지):
2. 추천 진료과(1줄):
3. 예방 및 관리 방법(2줄 이내):

[증상 정보]
{retrieved_context}
"""

# [6] SKT A.X-4.0 API로 답변 생성 (OpenAI 스타일)
client = OpenAI(
    base_url="https://guest-api.sktax.chat/v1",
    api_key="sktax-XyeKFrq67ZjS4EpsDlrHHXV8it"  # 여기에 발급받은 키를 넣으세요
)

response = client.chat.completions.create(
    model="ax4",  # skt/A.X-4.0 API 전용 모델명 (주의: "skt/A.X-4.0" 아님)
    messages=[
        {"role": "user", "content": prompt}
    ]
)
answer = response.choices[0].message.content.strip()

# [7] 번호 답변만 추출 (정규표현식)
matches = re.findall(r"1\..*?\n2\..*?\n3\..*(?=\n|$)", answer, flags=re.DOTALL)
answer_only = matches[-1].strip() if matches else answer

print("\n[AX-4.0 답변]\n")
display(Markdown(answer_only))


[AX-4.0 답변]



1. 감기, 부비동염, 기관지염:
2. 이비인후과, 호흡기내과:
3. 충분한 수분 섭취, 휴식 및 실내 습도 유지: